# Aggregated H2-Electrolyser Potentials

In [1]:
import pandas as pd
import numpy as np
import os
import glob
from scipy import stats

In [121]:
# Define the directory containing the CSV files
input_directory = r'D:\Fichtner GmbH & Co. KG\FIS_F_Value_Chain_Model - General\400_Implementation\b_production\3-Global Electrolyser_Potential_PtX_Atlas_Input\1-Input_Raw_Files'
output_directory = r'D:\Fichtner GmbH & Co. KG\FIS_F_Value_Chain_Model - General\400_Implementation\b_production\3-Global Electrolyser_Potential_PtX_Atlas_Input'

# Get a list of all CSV files in the directory
csv_files = glob.glob(os.path.join(input_directory, "*.csv"))
csv_files

# Initialize a list to store the results
results = []
global_results = []

# Specify the data types for columns that should be floats
dtype_dict = {'pv': float, 'wind': float, 'electrolysis': float}




for file_path in csv_files:
    try:
        print(f"Processing file: {file_path}")
        
        # Extract information from file name
        base_name = os.path.basename(file_path)  # Get the file name from the file path
        name_parts = base_name.split('_')  # Split the file name at underscores
        parameter = name_parts[1]  # First word: "electrolyser"
        location = name_parts[2]  # Abbreviation "CHL"
        region = name_parts[-1] .split('.')[0]

        

        
        
        # Load each CSV file into a DataFrame
        # Load each CSV file into a DataFrame with specified data types
        df = pd.read_csv(file_path, sep=';', comment='#', engine='python', encoding='utf-8', dtype=dtype_dict)

        # Convert the time column to datetime format
        df['time'] = pd.to_datetime(df['time'])

        # Set 'time' as the index
        df.set_index('time', inplace=True)

        # Calculate the yearly sum for the electrolysis column
        yearly_sum_pv = df['pv'].resample('Y').sum().values[0]
        yearly_sum_wind = df['wind'].resample('Y').sum().values[0]
        yearly_sum_electrolyser = df['electrolysis'].resample('Y').sum().values[0]

        # Calculate the quarterly sums for the electrolysis column
        quarterly_sums_el = df['electrolysis'].resample('Q').sum()  /  (1e6)
        quarterly_sums_wind = df['wind'].resample('Q').sum()  /  (1e6)
        quarterly_sums_pv = df['wind'].resample('Q').sum()  /  (1e6)

        quarter_index = ['Q' + str(i.quarter) + ' ' + str(i.year) for i in quarterly_sums.index]
        
        
        # Generate separate lists for quarters and years
        quarters = ['Q' + str(i.quarter) for i in quarterly_sums_el.index]
        weather_years = [str(i.year) for i in quarterly_sums_el.index]


        # Add the results to the list
        result_el = pd.DataFrame({
            "process":"El",
            "location":location,
            "region":region,
            "parameter":"CapacityFactor",
            "Unit":"%",
            "quarter_index": quarters,
            'Value': quarterly_sums_el.values,
            })
        result_el['source'] = os.path.basename(file_path)
        result_el['Comment'] = "Global Power-to-X Atlas"
        global_results.append(result_el)
        
        
        result_wind = pd.DataFrame({
            "process": "Wind",
            "location": location,
            "region": region,
            "parameter": "CapacityFactor_Wind",
            "Unit": "%",
            "quarter_index": quarters,
            'Value': quarterly_sums_wind.values,
        })
        result_wind['source'] = os.path.basename(file_path)
        result_wind['Comment'] = "Global Power-to-X Atlas"
        global_results.append(result_wind) 
        
    
        # Add the results to the list for PV
        result_pv = pd.DataFrame({
            "process": "SolarPv",
            "location": location,
            "region": region,
            "parameter": "CapacityFactor_PV",
            "Unit": "%",
            "quarter_index": quarters,
            'Value': quarterly_sums_pv.values,
        })
        result_pv['source'] = os.path.basename(file_path)
        result_pv['Comment'] = "Global Power-to-X Atlas"
        global_results.append(result_pv)
        
        
        # Drop the 'Fuel' column for specific parameters
        drop_fuel_params = ['capex', 'FOM', 'VOM', 'lifetime']
        
        if group_keys[group_columns.index('parameter')] in drop_fuel_params:
            group_df = group_df.drop(columns=['Fuel'])

    except Exception as e:
        print(f"An error occurred while processing {file_path}: {e}")

# Concatenate all results into a single DataFrame
global_results_df = pd.concat(global_results, ignore_index=True)
global_results_df["Value"] = global_results_df["Value"].apply(lambda x: round(x,4))
global_results_df['process'] = global_results_df['process'].replace('electrolyser', 'El')
# Save the final results to a CSV file
final_output_file = os.path.join(output_directory, "capacity_factors.csv",)
global_results_df.to_csv(final_output_file, index=False,float_format='%.4f',sep=';', decimal=',')

print(f"All files have been processed. Results saved to {final_output_file}.")


Processing file: D:\Fichtner GmbH & Co. KG\FIS_F_Value_Chain_Model - General\400_Implementation\b_production\3-Global Electrolyser_Potential_PtX_Atlas_Input\1-Input_Raw_Files\_potential_ang_namibe.csv
An error occurred while processing D:\Fichtner GmbH & Co. KG\FIS_F_Value_Chain_Model - General\400_Implementation\b_production\3-Global Electrolyser_Potential_PtX_Atlas_Input\1-Input_Raw_Files\_potential_ang_namibe.csv: name 'group_keys' is not defined
Processing file: D:\Fichtner GmbH & Co. KG\FIS_F_Value_Chain_Model - General\400_Implementation\b_production\3-Global Electrolyser_Potential_PtX_Atlas_Input\1-Input_Raw_Files\_potential_AUS_westernaustralia.csv


C:\Users\MarxA\AppData\Local\Temp\ipykernel_22840\258361536.py:45: FutureWarning: 'Y' is deprecated and will be removed in a future version, please use 'YE' instead.
  yearly_sum_pv = df['pv'].resample('Y').sum().values[0]
C:\Users\MarxA\AppData\Local\Temp\ipykernel_22840\258361536.py:46: FutureWarning: 'Y' is deprecated and will be removed in a future version, please use 'YE' instead.
  yearly_sum_wind = df['wind'].resample('Y').sum().values[0]
C:\Users\MarxA\AppData\Local\Temp\ipykernel_22840\258361536.py:47: FutureWarning: 'Y' is deprecated and will be removed in a future version, please use 'YE' instead.
  yearly_sum_electrolyser = df['electrolysis'].resample('Y').sum().values[0]
C:\Users\MarxA\AppData\Local\Temp\ipykernel_22840\258361536.py:50: FutureWarning: 'Y' is deprecated and will be removed in a future version, please use 'YE' instead.
  quarterly_sums_el = df['electrolysis'].resample('Y').sum()  /  (1e6)
C:\Users\MarxA\AppData\Local\Temp\ipykernel_22840\258361536.py:51: Fut

An error occurred while processing D:\Fichtner GmbH & Co. KG\FIS_F_Value_Chain_Model - General\400_Implementation\b_production\3-Global Electrolyser_Potential_PtX_Atlas_Input\1-Input_Raw_Files\_potential_AUS_westernaustralia.csv: name 'group_keys' is not defined
Processing file: D:\Fichtner GmbH & Co. KG\FIS_F_Value_Chain_Model - General\400_Implementation\b_production\3-Global Electrolyser_Potential_PtX_Atlas_Input\1-Input_Raw_Files\_potential_BRA_amapa.csv
An error occurred while processing D:\Fichtner GmbH & Co. KG\FIS_F_Value_Chain_Model - General\400_Implementation\b_production\3-Global Electrolyser_Potential_PtX_Atlas_Input\1-Input_Raw_Files\_potential_BRA_amapa.csv: name 'group_keys' is not defined
Processing file: D:\Fichtner GmbH & Co. KG\FIS_F_Value_Chain_Model - General\400_Implementation\b_production\3-Global Electrolyser_Potential_PtX_Atlas_Input\1-Input_Raw_Files\_potential_BRA_riogrande.csv
An error occurred while processing D:\Fichtner GmbH & Co. KG\FIS_F_Value_Chain_Mo

C:\Users\MarxA\AppData\Local\Temp\ipykernel_22840\258361536.py:45: FutureWarning: 'Y' is deprecated and will be removed in a future version, please use 'YE' instead.
  yearly_sum_pv = df['pv'].resample('Y').sum().values[0]
C:\Users\MarxA\AppData\Local\Temp\ipykernel_22840\258361536.py:46: FutureWarning: 'Y' is deprecated and will be removed in a future version, please use 'YE' instead.
  yearly_sum_wind = df['wind'].resample('Y').sum().values[0]
C:\Users\MarxA\AppData\Local\Temp\ipykernel_22840\258361536.py:47: FutureWarning: 'Y' is deprecated and will be removed in a future version, please use 'YE' instead.
  yearly_sum_electrolyser = df['electrolysis'].resample('Y').sum().values[0]
C:\Users\MarxA\AppData\Local\Temp\ipykernel_22840\258361536.py:50: FutureWarning: 'Y' is deprecated and will be removed in a future version, please use 'YE' instead.
  quarterly_sums_el = df['electrolysis'].resample('Y').sum()  /  (1e6)
C:\Users\MarxA\AppData\Local\Temp\ipykernel_22840\258361536.py:51: Fut

An error occurred while processing D:\Fichtner GmbH & Co. KG\FIS_F_Value_Chain_Model - General\400_Implementation\b_production\3-Global Electrolyser_Potential_PtX_Atlas_Input\1-Input_Raw_Files\_potential_BRA_riograndedosul.csv: name 'group_keys' is not defined
Processing file: D:\Fichtner GmbH & Co. KG\FIS_F_Value_Chain_Model - General\400_Implementation\b_production\3-Global Electrolyser_Potential_PtX_Atlas_Input\1-Input_Raw_Files\_potential_CAN_newfoundland.csv
An error occurred while processing D:\Fichtner GmbH & Co. KG\FIS_F_Value_Chain_Model - General\400_Implementation\b_production\3-Global Electrolyser_Potential_PtX_Atlas_Input\1-Input_Raw_Files\_potential_CAN_newfoundland.csv: name 'group_keys' is not defined
Processing file: D:\Fichtner GmbH & Co. KG\FIS_F_Value_Chain_Model - General\400_Implementation\b_production\3-Global Electrolyser_Potential_PtX_Atlas_Input\1-Input_Raw_Files\_potential_CHL_atacama.csv
An error occurred while processing D:\Fichtner GmbH & Co. KG\FIS_F_Valu

C:\Users\MarxA\AppData\Local\Temp\ipykernel_22840\258361536.py:45: FutureWarning: 'Y' is deprecated and will be removed in a future version, please use 'YE' instead.
  yearly_sum_pv = df['pv'].resample('Y').sum().values[0]
C:\Users\MarxA\AppData\Local\Temp\ipykernel_22840\258361536.py:46: FutureWarning: 'Y' is deprecated and will be removed in a future version, please use 'YE' instead.
  yearly_sum_wind = df['wind'].resample('Y').sum().values[0]
C:\Users\MarxA\AppData\Local\Temp\ipykernel_22840\258361536.py:47: FutureWarning: 'Y' is deprecated and will be removed in a future version, please use 'YE' instead.
  yearly_sum_electrolyser = df['electrolysis'].resample('Y').sum().values[0]
C:\Users\MarxA\AppData\Local\Temp\ipykernel_22840\258361536.py:50: FutureWarning: 'Y' is deprecated and will be removed in a future version, please use 'YE' instead.
  quarterly_sums_el = df['electrolysis'].resample('Y').sum()  /  (1e6)
C:\Users\MarxA\AppData\Local\Temp\ipykernel_22840\258361536.py:51: Fut

An error occurred while processing D:\Fichtner GmbH & Co. KG\FIS_F_Value_Chain_Model - General\400_Implementation\b_production\3-Global Electrolyser_Potential_PtX_Atlas_Input\1-Input_Raw_Files\_potential_MAR_laayoune.csv: name 'group_keys' is not defined
Processing file: D:\Fichtner GmbH & Co. KG\FIS_F_Value_Chain_Model - General\400_Implementation\b_production\3-Global Electrolyser_Potential_PtX_Atlas_Input\1-Input_Raw_Files\_potential_SAU_tabuk.csv
An error occurred while processing D:\Fichtner GmbH & Co. KG\FIS_F_Value_Chain_Model - General\400_Implementation\b_production\3-Global Electrolyser_Potential_PtX_Atlas_Input\1-Input_Raw_Files\_potential_SAU_tabuk.csv: name 'group_keys' is not defined
Processing file: D:\Fichtner GmbH & Co. KG\FIS_F_Value_Chain_Model - General\400_Implementation\b_production\3-Global Electrolyser_Potential_PtX_Atlas_Input\1-Input_Raw_Files\_potential_USA_florida.csv
An error occurred while processing D:\Fichtner GmbH & Co. KG\FIS_F_Value_Chain_Model - Gene

C:\Users\MarxA\AppData\Local\Temp\ipykernel_22840\258361536.py:45: FutureWarning: 'Y' is deprecated and will be removed in a future version, please use 'YE' instead.
  yearly_sum_pv = df['pv'].resample('Y').sum().values[0]
C:\Users\MarxA\AppData\Local\Temp\ipykernel_22840\258361536.py:46: FutureWarning: 'Y' is deprecated and will be removed in a future version, please use 'YE' instead.
  yearly_sum_wind = df['wind'].resample('Y').sum().values[0]
C:\Users\MarxA\AppData\Local\Temp\ipykernel_22840\258361536.py:47: FutureWarning: 'Y' is deprecated and will be removed in a future version, please use 'YE' instead.
  yearly_sum_electrolyser = df['electrolysis'].resample('Y').sum().values[0]
C:\Users\MarxA\AppData\Local\Temp\ipykernel_22840\258361536.py:50: FutureWarning: 'Y' is deprecated and will be removed in a future version, please use 'YE' instead.
  quarterly_sums_el = df['electrolysis'].resample('Y').sum()  /  (1e6)
C:\Users\MarxA\AppData\Local\Temp\ipykernel_22840\258361536.py:51: Fut

In [122]:
global_results_df["Value"].max()

np.float64(2.0592)

In [123]:
global_results_df[global_results_df["region"]=="notherncape"]

,process,location,region,parameter,Unit,quarter_index,Value,source,Comment
33,El,ZAF,notherncape,CapacityFactor,%,Q4,1.8462,_potential_ZAF_notherncape.csv,Global Power-to-X Atlas
34,Wind,ZAF,notherncape,CapacityFactor_Wind,%,Q4,0.0025,_potential_ZAF_notherncape.csv,Global Power-to-X Atlas
35,SolarPv,ZAF,notherncape,CapacityFactor_PV,%,Q4,0.0025,_potential_ZAF_notherncape.csv,Global Power-to-X Atlas
